## Imports

In [1]:
import math
import numpy as np
import pandas
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score

# Preparando el dataset de entrenamiento

In [2]:
data_train = pandas.read_csv('./data/perrosTrainData.csv')

data_train

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,1,Adulto,Mediano,NaN,Bicolor,Blanco,Negro,NaN,Corto,NaN,Corto,NaN,Cortas,NaN
1,2,Cachorro,Mediano,NaN,Bicolor,Blanco,Marron,NaN,NaN,NaN,Largo,NaN,Cortas,Caidas
2,3,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,NaN,Largas,NaN
3,4,Adulto,Mediano,NaN,NaN,NaN,NaN,NaN,Corto,NaN,Largo,NaN,NaN,Paradas
4,5,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
5,6,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
6,7,NaN,Mediano,NaN,NaN,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,NaN
7,8,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,NaN,Caidas
8,9,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas
9,10,Adulto,Grande,NaN,Liso,NaN,NaN,NaN,Mediano,Marron oscuro,Largo,Largo,Largas,NaN


### Obtenemos la cantidad de valores vacios

In [3]:
null_value_stats = data_train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Edad                  9
Sexo                 20
Patron de pelaje      8
Color de pelaje 1     7
Color de pelaje 2    21
Color de pelaje 3    25
Largo de pelaje       4
Color de ojos        10
Largo de hocico       4
Largo de cola         9
Largo de orejas       8
Tipo de orejas       11
dtype: int64

### Lleno los nulos con strings vacios y quito la columna "Mascota"

In [4]:
data_train.fillna("NaN", inplace=True)
prepared_data_train = data_train.drop('Mascota', axis=1)
prepared_data_train

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,Mediano,NaN,Bicolor,Blanco,Negro,NaN,Corto,NaN,Corto,NaN,Cortas,NaN
1,Cachorro,Mediano,NaN,Bicolor,Blanco,Marron,NaN,NaN,NaN,Largo,NaN,Cortas,Caidas
2,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,NaN,Largas,NaN
3,Adulto,Mediano,NaN,NaN,NaN,NaN,NaN,Corto,NaN,Largo,NaN,NaN,Paradas
4,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
5,NaN,Chico,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,Caidas
6,NaN,Mediano,NaN,NaN,Negro,NaN,NaN,Corto,Marron oscuro,Corto,NaN,Cortas,NaN
7,Adulto,Mediano,NaN,Liso,Negro,NaN,NaN,Corto,Marron oscuro,Largo,Largo,NaN,Caidas
8,Cachorro,Chico,NaN,Bicolor,Blanco,NaN,NaN,Corto,NaN,NaN,Corto,Cortas,Caidas
9,Adulto,Grande,NaN,Liso,NaN,NaN,NaN,Mediano,Marron oscuro,Largo,Largo,Largas,NaN


### Imprimo los tipos de cada columna

In [5]:
print(prepared_data_train.dtypes)


Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


### Obtengo la lista de etiquetas de las categorias

In [6]:
dataset_labels = prepared_data_train.columns.to_list()
dataset_labels

['Edad',
 'Tamaño',
 'Sexo',
 'Patron de pelaje',
 'Color de pelaje 1',
 'Color de pelaje 2',
 'Color de pelaje 3',
 'Largo de pelaje',
 'Color de ojos',
 'Largo de hocico',
 'Largo de cola',
 'Largo de orejas',
 'Tipo de orejas']

In [7]:
data_train_mascota_column = data_train.Mascota
data_train_mascota_column.head()

0    1
1    2
2    3
3    4
4    5
Name: Mascota, dtype: int64

Obtengo las categorias en base a las que no son numericas, en este caso todas son categorias

In [8]:
categorical_features_indices = np.where(prepared_data_train.dtypes != np.float)[0]
categorical_features_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

# Preparando el dataset de testeo

In [10]:
dataset_test = pandas.read_csv('./data/perrosTestData.csv')
dataset_test.head()

,Mascota,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,26,Adulto,NaN,Macho,Bicolor,Negro,Blanco,NaN,Corto,Marron oscuro,Largo,Corto,Mediano,Caidas
1,27,Adulto,Grande,NaN,Tricolor,Negro,Marron,Blanco,Corto,Marron oscuro,NaN,NaN,Mediano,Caidas
2,28,Adulto,Mediano,NaN,Liso,NaN,NaN,NaN,Mediano,NaN,NaN,Largo,Cortas,Caidas
3,29,Cachorro,Chico,NaN,Liso,NaN,NaN,NaN,Corto,Marron oscuro,NaN,Largo,NaN,Caidas
4,30,NaN,NaN,NaN,Liso,Negro,NaN,NaN,Mediano,NaN,Largo,NaN,Mediano,Paradas


In [11]:
null_value_stats = dataset_test.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Edad                  9
Tamaño                5
Sexo                  9
Patron de pelaje      4
Color de pelaje 1     5
Color de pelaje 2    13
Color de pelaje 3    23
Largo de pelaje       5
Color de ojos        13
Largo de hocico      12
Largo de cola         6
Largo de orejas       8
Tipo de orejas       10
dtype: int64

In [12]:
dataset_test.fillna("NaN", inplace=True)
prepared_dataset_test = dataset_test.drop('Mascota', axis=1)
prepared_dataset_test.head()

,Edad,Tamaño,Sexo,Patron de pelaje,Color de pelaje 1,Color de pelaje 2,Color de pelaje 3,Largo de pelaje,Color de ojos,Largo de hocico,Largo de cola,Largo de orejas,Tipo de orejas
0,Adulto,NaN,Macho,Bicolor,Negro,Blanco,NaN,Corto,Marron oscuro,Largo,Corto,Mediano,Caidas
1,Adulto,Grande,NaN,Tricolor,Negro,Marron,Blanco,Corto,Marron oscuro,NaN,NaN,Mediano,Caidas
2,Adulto,Mediano,NaN,Liso,NaN,NaN,NaN,Mediano,NaN,NaN,Largo,Cortas,Caidas
3,Cachorro,Chico,NaN,Liso,NaN,NaN,NaN,Corto,Marron oscuro,NaN,Largo,NaN,Caidas
4,NaN,NaN,NaN,Liso,Negro,NaN,NaN,Mediano,NaN,Largo,NaN,Mediano,Paradas


In [13]:
print(prepared_dataset_test.dtypes)

Edad                 object
Tamaño               object
Sexo                 object
Patron de pelaje     object
Color de pelaje 1    object
Color de pelaje 2    object
Color de pelaje 3    object
Largo de pelaje      object
Color de ojos        object
Largo de hocico      object
Largo de cola        object
Largo de orejas      object
Tipo de orejas       object
dtype: object


In [15]:
dataset_test_mascota_column = dataset_test.Mascota
dataset_test_mascota_column.head()

0    26
1    27
2    28
3    29
4    30
Name: Mascota, dtype: int64

# Creando y entrenando el modelo

### Creacion del modelo

In [16]:
eval_dataset = Pool(data=prepared_dataset_test,
                    label=dataset_test_mascota_column,
                    cat_features=categorical_features_indices)

In [17]:
train_dataset = Pool(data=prepared_data_train,
                        label=data_train_mascota_column,
                        cat_features=categorical_features_indices)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000,
                            learning_rate=1,
                            depth=5,
                            loss_function='MultiClass')
# Fit model


### Entrenamiento

In [18]:
model.fit(train_dataset)


0:	learn: 3.1326313	total: 114ms	remaining: 1m 53s
1:	learn: 2.8399750	total: 339ms	remaining: 2m 49s
2:	learn: 2.6750906	total: 572ms	remaining: 3m 10s
3:	learn: 2.4938895	total: 669ms	remaining: 2m 46s
4:	learn: 2.2585422	total: 1.04s	remaining: 3m 26s
5:	learn: 2.0395610	total: 1.35s	remaining: 3m 44s
6:	learn: 1.9433741	total: 1.66s	remaining: 3m 55s
7:	learn: 1.7723055	total: 1.98s	remaining: 4m 5s
8:	learn: 1.5917711	total: 2.36s	remaining: 4m 20s
9:	learn: 1.4266755	total: 2.73s	remaining: 4m 30s
10:	learn: 1.3804444	total: 3.21s	remaining: 4m 48s
11:	learn: 1.2725092	total: 3.67s	remaining: 5m 2s
12:	learn: 1.1579400	total: 3.96s	remaining: 5m
13:	learn: 1.0892704	total: 4.25s	remaining: 4m 59s
14:	learn: 1.0122719	total: 4.53s	remaining: 4m 57s
15:	learn: 0.9109881	total: 4.9s	remaining: 5m 1s
16:	learn: 0.8688988	total: 5.27s	remaining: 5m 4s
17:	learn: 0.7936213	total: 5.71s	remaining: 5m 11s
18:	learn: 0.7526793	total: 6.04s	remaining: 5m 11s
19:	learn: 0.7048985	total: 6.3

Generar predicciones

In [19]:
# Get predicted classes
preds_class = model.predict(eval_dataset)
# Get predicted probabilities for each class
preds_proba = model.predict_proba(eval_dataset)
# Get predicted RawFormulaVal
preds_raw = model.predict(eval_dataset, prediction_type='RawFormulaVal')

In [20]:
print(preds_class)

[[25]
 [ 6]
 [16]
 [16]
 [ 6]
 [ 6]
 [15]
 [16]
 [25]
 [20]
 [22]
 [15]
 [24]
 [11]
 [ 6]
 [19]
 [11]
 [13]
 [ 6]
 [20]
 [24]
 [21]
 [24]
 [20]
 [21]]


In [21]:
print(preds_proba)

[[0.02955369 0.06911398 0.03810611 0.03877622 0.01717751 0.08293271
  0.01735112 0.04783695 0.0761681  0.03414299 0.01212847 0.03582176
  0.01506557 0.02146213 0.01779038 0.01141718 0.06099657 0.02000849
  0.03890339 0.09549184 0.02446095 0.04814199 0.02356241 0.01327817
  0.11031129]
 [0.03004688 0.01141748 0.00460485 0.01331431 0.0329022  0.35413084
  0.07243422 0.00631052 0.00369895 0.01100003 0.03236801 0.02007226
  0.08715346 0.00632729 0.04170452 0.16183854 0.00935085 0.01395159
  0.00659445 0.00738096 0.01460356 0.01351583 0.0100666  0.02733408
  0.00787772]
 [0.04147942 0.01359523 0.00917024 0.00905298 0.04111813 0.03660349
  0.0446921  0.00439674 0.00542852 0.03969441 0.03925592 0.01074939
  0.01467326 0.00724656 0.01559316 0.5533371  0.00409833 0.00717838
  0.00563133 0.0091032  0.00759959 0.04678474 0.01171148 0.01541566
  0.00639063]
 [0.01952596 0.00899446 0.00489006 0.00496867 0.05731488 0.23608889
  0.04198086 0.00507863 0.00270671 0.01267597 0.04091718 0.01035408
  0.01

In [22]:
print(preds_raw)

[[-8.50206685e-02  7.64527709e-01  1.69145370e-01  1.86577808e-01
  -6.27628564e-01  9.46800210e-01 -6.17572594e-01  3.96569059e-01
   8.61713335e-01  5.93279667e-02 -9.75673631e-01  1.07326125e-01
  -7.58817355e-01 -4.04939306e-01 -5.92571269e-01 -1.03610985e+00
   6.39588319e-01 -4.75072558e-01  1.89851970e-01  1.08781143e+00
  -2.74151440e-01  4.02925377e-01 -3.11576565e-01 -8.85107800e-01
   1.23207692e+00]
 [ 5.03683699e-01 -4.63929416e-01 -1.37196440e+00 -3.10235655e-01
   5.94464411e-01  2.97059117e+00  1.38360357e+00 -1.05685790e+00
  -1.59102513e+00 -5.01177635e-01  5.78095175e-01  1.00263455e-01
   1.56859515e+00 -1.05420259e+00  8.31534185e-01  2.18752390e+00
  -6.63608397e-01 -2.63481670e-01 -1.01284709e+00 -9.00171936e-01
  -2.17809852e-01 -2.95213449e-01 -5.89852150e-01  4.09058984e-01
  -8.35036436e-01]
 [ 9.75538597e-01 -1.39940069e-01 -5.33695218e-01 -5.46565407e-01
   9.66790142e-01  8.50484719e-01  1.05013797e+00 -1.26879545e+00
  -1.05799207e+00  9.31551448e-01  9.2

### Supuestas predicciones

In [23]:
mascota_numero = 0

for prediccion in preds_class:
    probabilidad = math.trunc((preds_proba[mascota_numero][prediccion.item()-1])*100)
    print("-- Mascota "+str(dataset_test_mascota_column[mascota_numero])+" es similar a la mascota "+str(prediccion.item())
        +" => "+str(probabilidad)+"% de probabilidad")
    # print(str(preds_proba[mascota_numero]))
    # print(dataset_test.loc[mascota_numero:mascota_numero, "Edad":"Tipo de orejas"].values)
    # print(prepared_data_train.loc[prediccion.item():prediccion.item(), "Edad":"Tipo de orejas"].values)
    mascota_numero+=1


-- Mascota 26 es similar a la mascota 25 => 11% de probabilidad
-- Mascota 27 es similar a la mascota 6 => 35% de probabilidad
-- Mascota 28 es similar a la mascota 16 => 55% de probabilidad
-- Mascota 29 es similar a la mascota 16 => 43% de probabilidad
-- Mascota 30 es similar a la mascota 6 => 35% de probabilidad
-- Mascota 31 es similar a la mascota 6 => 25% de probabilidad
-- Mascota 32 es similar a la mascota 15 => 34% de probabilidad
-- Mascota 33 es similar a la mascota 16 => 21% de probabilidad
-- Mascota 34 es similar a la mascota 25 => 11% de probabilidad
-- Mascota 35 es similar a la mascota 20 => 60% de probabilidad
-- Mascota 36 es similar a la mascota 22 => 12% de probabilidad
-- Mascota 37 es similar a la mascota 15 => 42% de probabilidad
-- Mascota 38 es similar a la mascota 24 => 33% de probabilidad
-- Mascota 39 es similar a la mascota 11 => 19% de probabilidad
-- Mascota 40 es similar a la mascota 6 => 32% de probabilidad
-- Mascota 41 es similar a la mascota 19 => 